In [3]:
!pip install python-dotenv


  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
Using cached python_dotenv-1.2.1-py3-none-any.whl (21 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: C:\Users\jahna\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:

!{sys.executable} -m pip install python-dotenv


  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
Using cached python_dotenv-1.2.1-py3-none-any.whl (21 kB)


In [2]:
import sys
print(sys.executable)




c:\Users\jahna\OneDrive\Desktop\eeg+mri\EpilepsyNexus\venv\Scripts\python.exe


In [3]:
!{sys.executable} -m pip install langchain-groq

  Using cached langchain_groq-1.1.2-py3-none-any.whl.metadata (2.4 kB)
  Using cached groq-0.37.1-py3-none-any.whl.metadata (16 kB)
  Using cached langchain_core-1.2.9-py3-none-any.whl.metadata (4.4 kB)
  Using cached anyio-4.12.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached langsmith-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached tenacity-9.1.4-py3-none-any.whl.metadata (1.2 kB)
  Using cached uuid_utils-0.14.0-cp39-abi3-win_amd64.whl.metadata (5.0 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached certifi-2026.1.4-py3-none-any.whl.metadata (2.5 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached

In [4]:
!{sys.executable} -m pip install langchain



  Using cached langchain-1.2.9-py3-none-any.whl.metadata (5.7 kB)
  Using cached langgraph-1.0.8-py3-none-any.whl.metadata (7.4 kB)
  Using cached langgraph_checkpoint-4.0.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached langgraph_prebuilt-1.0.7-py3-none-any.whl.metadata (5.2 kB)
  Using cached langgraph_sdk-0.3.4-py3-none-any.whl.metadata (1.6 kB)
Using cached langchain-1.2.9-py3-none-any.whl (111 kB)
Using cached langgraph-1.0.8-py3-none-any.whl (158 kB)
Using cached langgraph_checkpoint-4.0.0-py3-none-any.whl (46 kB)
Using cached langgraph_prebuilt-1.0.7-py3-none-any.whl (35 kB)
Using cached langgraph_sdk-0.3.4-py3-none-any.whl (67 kB)
   ---------------------------------------- 0.0/117.2 kB ? eta -:--:--
   ---------- ---------------------------- 30.7/117.2 kB 640.0 kB/s eta 0:00:01
   ---------------------------------------- 117.2/117.2 kB 1.4 MB/s eta 0:00:00


In [2]:
import os
import json
from dotenv import load_dotenv
from langchain_groq import ChatGroq
# from langchain.prompts import PromptTemplate
from langchain_core.prompts import PromptTemplate

from graph import EpilepsyState

load_dotenv()

True

In [ ]:
# llm = ChatGroq(
#     model="llama-3.1-8b-instant",
#     api_key=os.getenv("GROQ_API_KEY"),
#     temperature=0.1,
# )


In [13]:
# prompt = PromptTemplate(
#     template=open("prompts/mri_eeg_fusion.txt").read(),
#     input_variables=[
#         "mri_epilepsy_label",
#         "mri_confidence",
#         "seizure_phase",
#         "seizure_type",
#     ],
# )
from pathlib import Path
from langchain_core.prompts import PromptTemplate

BASE_DIR = Path.cwd()  # EpilepsyNexus
prompt_path = BASE_DIR / "prompts" / "mri_eeg_combiner_prompt.txt"

prompt = PromptTemplate(
    template=prompt_path.read_text(encoding="utf-8"),
    input_variables=[
        "mri_epilepsy_label",
        "mri_confidence",
        "seizure_phase",
        "seizure_type",
    ],
)


In [6]:




def mri_eeg_fusion_node(state: EpilepsyState) -> EpilepsyState:
    """
    Combines MRI and EEG classifier outputs using Groq LLM.
    Produces epilepsy_presence + fusion explanation.
    """

    # ---------- Read from state ----------
    inputs = {
        "mri_epilepsy_label": state.mri_epilepsy_label,
        "mri_confidence": state.mri_confidence,
        "seizure_phase": state.seizure_phase,
        "seizure_type": state.seizure_type,
    }

    # ---------- Run LLM ----------
    response = llm.invoke(prompt.format(**inputs))

    # ---------- Parse JSON safely ----------
    try:
        output = json.loads(response.content)
    except json.JSONDecodeError:
        state.epilepsy_presence = "uncertain"
        state.fusion_explanation = (
            "Fusion agent could not reliably combine MRI and EEG findings."
        )
        return state

    # ---------- Write back to state ----------
    state.epilepsy_presence = output.get("epilepsy_presence")
    state.seizure_phase = output.get("seizure_phase")
    state.seizure_type = output.get("seizure_type")
    state.fusion_explanation = output.get("fusion_explanation")

    return state

In [14]:
formatted_prompt = prompt.format(
    mri_epilepsy_label="Temporal Lobe Epilepsy",
    mri_confidence="0.92",
    seizure_phase="Ictal",
    seizure_type="Focal seizure"
)

print(formatted_prompt)


You are a medical AI assistant for epilepsy analysis.

STRICT RULES:
- Do NOT diagnose.
- Do NOT recommend medication or treatment.
- Do NOT give probabilities unless provided.
- Only reason based on the given MRI and EEG outputs.
- If evidence is insufficient or conflicting, say "uncertain".

INPUTS:

MRI Result:
- Classification: Temporal Lobe Epilepsy
- Confidence: 0.92

EEG Result:
- Seizure Phase: Ictal
- Seizure Type: Focal seizure

TASK:
1. Decide epilepsy_presence as one of:
   - "yes"
   - "no"
   - "uncertain"

2. Keep seizure_phase and seizure_type ONLY if epilepsy_presence is "yes".
   Otherwise, return "not_applicable".

3. Write a concise medical fusion explanation
   explaining how MRI and EEG findings relate.

OUTPUT FORMAT (STRICT JSON):
{
  "epilepsy_presence": "",
  "seizure_phase": "",
  "seizure_type": "",
  "fusion_explanation": ""
}



In [8]:
print(prompt.template[:300])


You are a medical AI assistant for epilepsy analysis.

STRICT RULES:
- Do NOT diagnose.
- Do NOT recommend medication or treatment.
- Do NOT give probabilities unless provided.
- Only reason based on the given MRI and EEG outputs.
- If evidence is insufficient or conflicting, say "uncertain".

INPUT


In [9]:
state = EpilepsyState(
    mri_epilepsy_label="epilepsy",
    mri_confidence=0.93,
    seizure_phase="ictal",
    seizure_type="focal",
)


In [16]:
state = mri_eeg_fusion_node(state)


print(json.dumps(state.model_dump(), indent=4))


{
    "mri_image_path": null,
    "eeg_text_file_path": null,
    "mri_epilepsy_label": "epilepsy",
    "mri_confidence": 0.93,
    "seizure_phase": "ictal",
    "seizure_type": "focal",
    "epilepsy_presence": "yes",
    "fusion_explanation": "The MRI classification of epilepsy with high confidence (0.93) supports the presence of epilepsy. The EEG result indicating an ictal seizure phase and focal seizure type further confirms the diagnosis, suggesting a localized seizure activity consistent with epilepsy.",
    "medical_context": null,
    "neuro_diagnostic_report": null,
    "patient_explanation": null,
    "safety_passed": false,
    "safety_notes": null
}


In [1]:
import json
import sys
from pathlib import Path
from langchain_core.prompts import PromptTemplate


PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))
print(PROJECT_ROOT)

from graph import EpilepsyState
from llms.groq_llm import get_groq_llm


PROJECT_ROOT = Path.cwd().parent  
prompt_path = PROJECT_ROOT / "prompts" / "neuro_diagnostic_report_prompt.txt"
with open(prompt_path, "r", encoding="utf-8") as f:
    prompt = f.read()


llm = get_groq_llm()

c:\Users\jahna\OneDrive\Desktop\eeg+mri\EpilepsyNexus
